In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


In [4]:
import env 
import acquire
import prepare
import model

# ACQUIRE

In [ ]:
raw_df = acquire.get_telco_chunk()

In [ ]:
raw_df.info()

In [ ]:
raw_df = raw_df.set_index('customer_id')

In [2]:
def encode_variable(column, df):
    lab_enc = LabelEncoder()
    lab_enc.fit(df[column])
    df[column] = lab_enc.transform(df[column])

In [ ]:
encode_variable('gender', raw_df)

In [ ]:
cat_cols = raw_df.select_dtypes('object').columns

In [ ]:
for i in cat_cols:
    print(i)
    encode_variable(i, df)

In [ ]:
raw_df.head()

### It's acquired, now
# PREPARE

In [5]:
df = prepare.prep_telco()

In [ ]:
# # Use a label encoder to transform the embarked column.
def encode_variable(column, df):
    lab_enc = LabelEncoder()
    lab_enc.fit(df[column])
    df[column] = lab_enc.transform(df[column])

In [8]:
cat_cols = df.select_dtypes('object').columns

In [11]:
for i in cat_cols:
    print(i)
    encode_variable(i, df)

gender
partner
dependents
phone_service
multiple_lines
online_security
online_backup
device_protection
tech_support
streaming_tv
streaming_movies
paperless_billing
churn


In [12]:
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
customer_id,,,,,,,,,,,,,,,,,,,,
0002-ORFBO,0,0,1,1,9,1,0,1,0,2,0,2,2,0,2,1,2,65.6,593.30,0
0003-MKNFE,1,0,0,0,9,1,2,1,0,0,0,0,0,2,1,0,2,59.9,542.40,0
0004-TLHLJ,1,0,0,0,4,1,0,2,0,0,2,0,0,0,1,1,1,73.9,280.85,1
0011-IGKFF,1,1,1,0,13,1,0,2,0,2,2,0,2,2,1,1,1,98.0,1237.85,1
0013-EXCHZ,0,1,1,0,3,1,0,2,0,0,0,2,2,0,1,1,2,83.9,267.40,1


## TEST/SPLIT X/y

In [ ]:
X = df.drop('churn', axis=1)
#X = df[['monthly_charges']]

In [ ]:
y = df['churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=123)

In [ ]:
X_train.head()

### PREP is boring, Let's 
# MODEL AND CHART STUFF. 
# OR EXPLORE. 
I guess.

In [ ]:
sns.countplot(x='churn', data=df)

In [ ]:
sns.countplot(x='churn', data=df, hue='dependents')

In [ ]:
#Boxplot to show the distribution of ages. X is class, Y is age.
plt.figure(figsize = (10,7))
sns.boxplot(x='churn', y='tenure', data=df)

# MODEL, Decision-Tree

In [ ]:
model.do_the_decisionTree('Entropy', X_train, y_train)

# MODEL, log-reg

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()

In [ ]:
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_train)
predictions_probas = logmodel.predict_proba(X_train)

In [ ]:
predictions_probas[:,0] < .5

In [ ]:
from sklearn import metrics

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_train, predictions)

In [ ]:
cnf_matrix

In [ ]:
#Looks like using logistic regression with just monthly charges as the predictor is worthless and predicts nobody
#will churn